## Load packages

In [1]:
import rdflib, glob, datetime, time
from rdflib import URIRef, Literal
from rdflib.namespace import RDF, RDFS, DCTERMS, SKOS, FOAF, VOID, XSD

## Define namespaces and utility functions

In [2]:
abr = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/")
abrc = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/4.0/classification/")
abri = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/4.0/instance/")
abrw = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/4.0/work/")
bf = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
arm = rdflib.Namespace("https://w3id.org/arm/core/ontology/0.1/")

In [3]:
def initialize_abr_graph():
    g = rdflib.Graph()
    g.bind("abr", "https://wsburroughs.link/anything-but-routine/")
    g.bind("abrc", "https://wsburroughs.link/anything-but-routine/4.0/classification/")
    g.bind("abri", "https://wsburroughs.link/anything-but-routine/4.0/instance/")
    g.bind("abrw", "https://wsburroughs.link/anything-but-routine/4.0/work/")
    g.bind("bf", "http://id.loc.gov/ontologies/bibframe/")
    g.bind("arm", "https://w3id.org/arm/core/ontology/0.1/")
    return g

In [4]:
def isonow():
    utc_offset_sec = time.altzone if time.localtime().tm_isdst else time.timezone
    utc_offset = datetime.timedelta(seconds=-utc_offset_sec)
    return datetime.datetime.now().replace(tzinfo=datetime.timezone(offset=utc_offset)).isoformat()

## Get graph for ABR v4.0

In [5]:
g = initialize_abr_graph()

for infile in glob.glob("docs/4.0/*/*.ttl"):
    g.parse(infile, format='n3')

## Update dcterms:modified,  void:triples,  void:entities,  void:rootResource for abr:

In [6]:
void_file = 'docs/void.ttl'

h = initialize_abr_graph()
h.parse(void_file, format='n3')

h.remove( (abr[''], DCTERMS.modified, None) )
h.add( (abr[''], DCTERMS.modified, Literal(isonow(), datatype=XSD.dateTime)) )

h.remove( (abr[''], VOID.triples, None) )
h.add( (abr[''], VOID.triples, Literal(len(g), datatype=XSD.integer)) )

n_entities = len([ cl for cl in g.subjects(RDF.type, bf.Classification) ])
n_entities += len([ w for w in g.subjects(RDF.type, bf.Work) ])
n_entities += len([ i for i in g.subjects(RDF.type, bf.Instance) ])
h.remove( (abr[''], VOID.entities, None) )
h.add( (abr[''], VOID.entities, Literal(n_entities, datatype=XSD.integer)) )

h.remove( (abr[''], VOID.rootResource, None) )
for work in g.subjects(RDF.type, bf.Work):
    h.add( (abr[''], VOID.rootResource, work) )

h.serialize(void_file, format='turtle')